In [3]:
%matplotlib ipympl

In [4]:
from impedance.preprocessing import readBioLogic
from impedance.models.circuits import CustomCircuit
import matplotlib.pyplot as plt
import numpy as np
from models import *
import mpmath as mp

In [5]:
#ALL THE FILES TO BE SEEN

name_prefix="RM_stacked second run"
list_of_name_sufix=["S3_1_C03.mpt","S3_2_C04.mpt","D12_1_C03.mpt","D12_2_C04.mpt"]

list_of_fnames=[]
for sfx in list_of_name_sufix:
    list_of_fnames.append(f"{name_prefix} {sfx}")
print(list_of_fnames)

['RM_stacked second run S3_1_C03.mpt', 'RM_stacked second run S3_2_C04.mpt', 'RM_stacked second run D12_1_C03.mpt', 'RM_stacked second run D12_2_C04.mpt']


In [7]:
import os
for fname in list_of_fnames:
    fpath=os.path.join("exp data","single type",fname)
    print(fpath)
    f,z=readBioLogic(f"../{fpath}")
    Rsep=100
    R1=1000
    Q=1e-4
    alpha=1
    c=CustomCircuit(initial_guess=[Rsep,R1,Q,alpha],circuit="R_1-TLMuni")


    # wt=np.concatenate((np.real(z),np.imag(z)))
    # print("hstack",wt_hstack)


    mp.dps=300
    c.fit(impedance=z,frequencies=f,bounds=((0,0,0,0.9),(np.inf,np.inf,np.inf,1)))
    
    z_fitted=c.predict(frequencies=f)
    print(c)

exp data\single type\RM_stacked second run S3_1_C03.mpt

Circuit string: R_1-TLMuni
Fit: True

Initial guesses:
    R_1 = 1.00e+02 [Ohm]
  TLMuni_0 = 1.00e+03 [Ohm]
  TLMuni_1 = 1.00e-04 [F]
  TLMuni_2 = 1.00e+00 []

Fit parameters:
    R_1 = 1.61e+02  (+/- 2.83e-01) [Ohm]
  TLMuni_0 = 4.99e+02  (+/- 1.22e+00) [Ohm]
  TLMuni_1 = 4.89e-03  (+/- 2.48e-06) [F]
  TLMuni_2 = 9.59e-01  (+/- 1.84e-04) []

exp data\single type\RM_stacked second run S3_2_C04.mpt

Circuit string: R_1-TLMuni
Fit: True

Initial guesses:
    R_1 = 1.00e+02 [Ohm]
  TLMuni_0 = 1.00e+03 [Ohm]
  TLMuni_1 = 1.00e-04 [F]
  TLMuni_2 = 1.00e+00 []

Fit parameters:
    R_1 = 1.46e+02  (+/- 2.83e-01) [Ohm]
  TLMuni_0 = 5.11e+02  (+/- 1.22e+00) [Ohm]
  TLMuni_1 = 4.94e-03  (+/- 2.52e-06) [F]
  TLMuni_2 = 9.62e-01  (+/- 1.85e-04) []

exp data\single type\RM_stacked second run D12_1_C03.mpt

Circuit string: R_1-TLMuni
Fit: True

Initial guesses:
    R_1 = 1.00e+02 [Ohm]
  TLMuni_0 = 1.00e+03 [Ohm]
  TLMuni_1 = 1.00e-04 [F]
  TL

e:\code\github.com\yugal0\fitting-experimental\code\models.py:13: RuntimeWarning: overflow encountered in tanh
  Z= 2*np.sqrt(R1/((omega*1j)**alpha*Q))*(1/(np.tanh(np.sqrt((1j*omega)**alpha*R1*Q))))
